In [0]:
pip install azure.eventhub faker

In [0]:
import json
import random
from datetime import datetime, timedelta
from faker import Faker
from azure.eventhub import EventHubProducerClient, EventData

# ----------------------------
# Azure Event Hub Config
# ----------------------------
CONNECTION_STR = dbutils.secrets.get(scope = "telco-secret-keys", key = "telco-payment-policy")
EVENTHUB_NAME = dbutils.secrets.get(scope = "telco-secret-keys", key = "telco-payment-hub")
# ----------------------------
# Faker Setup
# ----------------------------
fake = Faker()
CUSTOMER_IDS = list(range(1000, 1101))  # Customer IDs: 1000–1100
PAYMENT_METHODS = ["Credit Card", "Debit Card", "UPI", "Net Banking", "Wallet"]
PAYMENT_STATUS = ["Paid", "Pending", "Failed"]

# ----------------------------
# Generate Payment Record
# ----------------------------
def generate_payment(payment_id):
    customer_id = random.choice(CUSTOMER_IDS)
    amount = random.choice([199, 299, 499, 719, 799, 999, 1499])
    return {
        "PaymentID": f"P{payment_id}",
        "CustomerID": customer_id,
        "PaymentDate": (datetime.now() - timedelta(days=random.randint(0, 30))).strftime("%Y-%m-%d"),
        "Amount": amount,
        "Currency": "INR",
        "PaymentMethod": random.choice(PAYMENT_METHODS),
        "TransactionID": f"TXN{random.randint(100000,999999)}",
        "PaymentStatus": random.choice(PAYMENT_STATUS),
        "InvoiceID": f"INV{random.randint(5000,5999)}"
    }

# ----------------------------
# Send Events to Event Hub
# ----------------------------
def send_payment_events(num_events=50):
    producer = EventHubProducerClient.from_connection_string(
        conn_str=CONNECTION_STR,
        eventhub_name=EVENTHUB_NAME
    )
    event_batch = producer.create_batch()

    for i in range(num_events):
        payment_record = generate_payment(10000 + i)
        event_data = EventData(json.dumps(payment_record))
        event_batch.add(event_data)

    producer.send_batch(event_batch)
    print(f"✅ Sent {num_events} payment events to Event Hub")

# ----------------------------
# Main
# ----------------------------
if __name__ == "__main__":
    send_payment_events(50)  # send 50 dummy payment records
